In [5]:
import os
import json
from tencentfanyi import translate as t
from tencentfanyi import translate_test as t_test


In [8]:
# 提取日文，保存到jp.text
file_origial = os.listdir('input')
file_origial = list(map(lambda x:'input/'+x, file_origial))

jp = []
for file_name in file_origial:
    with open(file_name, 'r', encoding='utf-8') as f:
        text_t = f.readlines()
    for line_t in text_t:
        if line_t[-4:-1] in ('[p]', '[r]'):
            jp.append(line_t)

with open('intermediate_file/jp_all.txt', 'w', encoding='utf-8') as f:
    for line in jp:
        f.write(line)

In [10]:
# 根据文本建立空value字典，保存到 jp-chs.json
with open('intermediate_file/jp_all.txt', 'r', encoding='utf-8') as f:
    text_jp = f.readlines()
j_c = dict()
print('总共', len(text_jp), '条')
for line in text_jp:
    j_c[line[:-4]] = ''

with open('intermediate_file/jp-chs.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(j_c))



总共 2371 条


In [13]:
# 打开jp-chs.json文件检查
import json
with open('intermediate_file/jp-chs.json', 'r', encoding='utf-8') as f:
    js = json.loads(f.read())
with open('intermediate_file/jp_all.txt', 'r', encoding='utf-8') as f:
    text_jp = f.readlines()
print(list(js.keys())[:10])
print('需要翻译：', len(list(js.keys())), '条')
print('共有日文：', len(text_jp), '条， 应翻译：', len(set(text_jp)), '条')

['俺は今、最高に幸せだ。', '…富樫直哉、３２歳。普通のサラリーマン。', 'そして妻、理子。典型的な大和撫子の専業主婦。', '可愛い娘にも恵まれた、どこにでもいる、至極一般的な夫婦。', 'だけど…', '「理子…。愛してるよ」', '普通に愛し合う夫婦に見えるかもしれない。', 'だが、理子への過度ともいえる俺の愛情は、理子を寝取られるとより興奮するという…', '『一般的』『平凡』という言葉からは真逆のカタチを辿った…傍目では理解しがたい愛情で成り立っている夫婦、でもある。', '「理子…理子…あああ愛してるよ理子…」']
需要翻译： 2179 条
共有日文： 2371 条， 应翻译： 2179 条


In [16]:
# 把字典翻译
with open('intermediate_file/jp-chs.json', 'r', encoding='utf-8') as f:
    data = json.loads(f.read())

need_translated = len(list(data.keys()))
cnt = 0
cnt_last = 0
failed_list = []
for key in data:
    try:
        if not data[key]:
            ans = t(key)
            data[key] = ans if ans else key
        cnt+=1
        # FIXME 模拟间断
        # if cnt == 100:
        #     break
        if cnt - cnt_last == 10:
            cnt_last = cnt
            print(cnt, '/', need_translated)
            with open('intermediate_file/jp-chs.json', 'w', encoding='utf-8') as f:
                f.write(json.dumps(data))
    except Exception as e:
        print(e)
        failed_list.append(key)
else:
    with open('intermediate_file/jp-chs.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(data))
    with open('intermediate_file/failed.text', 'w', encoding='utf-8') as f:
        for line in failed_list:
            f.write(line)
    print('失败：', len(failed_list), '个！')


10 / 2179
20 / 2179
30 / 2179
40 / 2179
50 / 2179
60 / 2179
70 / 2179
80 / 2179
90 / 2179
100 / 2179
110 / 2179
120 / 2179
130 / 2179
140 / 2179
150 / 2179
160 / 2179
170 / 2179
180 / 2179
190 / 2179
200 / 2179
210 / 2179
220 / 2179
230 / 2179
240 / 2179
250 / 2179
260 / 2179
270 / 2179
280 / 2179
290 / 2179
300 / 2179
310 / 2179
320 / 2179
330 / 2179
340 / 2179
350 / 2179
360 / 2179
370 / 2179
380 / 2179
390 / 2179
400 / 2179
410 / 2179
420 / 2179
430 / 2179
440 / 2179
450 / 2179
460 / 2179
470 / 2179
480 / 2179
490 / 2179
500 / 2179
510 / 2179
520 / 2179
530 / 2179
540 / 2179
550 / 2179
560 / 2179
570 / 2179
580 / 2179
590 / 2179
600 / 2179
610 / 2179
620 / 2179
630 / 2179
640 / 2179
650 / 2179
660 / 2179
670 / 2179
680 / 2179
690 / 2179
700 / 2179
710 / 2179
720 / 2179
730 / 2179
740 / 2179
750 / 2179
760 / 2179
770 / 2179
780 / 2179
790 / 2179
800 / 2179
810 / 2179
820 / 2179
830 / 2179
840 / 2179
850 / 2179
860 / 2179
870 / 2179
880 / 2179
890 / 2179
900 / 2179
910 / 2179
920 / 21

In [17]:
# 构建对比
with open('intermediate_file/jp-chs.json', 'r', encoding='utf-8') as f:
    data = json.loads(f.read())
cnt = 0
with open('output/contrast.txt', 'w', encoding='utf-8') as f:
    for i in data:
        if data[i]:
            f.write(i+' '+data[i]+'\n')
        else:
            print(i, data[i])

In [64]:
# 替换原文件
with open('intermediate_file/jp-chs.json', 'r', encoding='utf-8') as f:
    jp_chs = json.loads(f.read())

file_origial = os.listdir('input')
failed_text = []

for file_name in file_origial:
    with open('input/'+file_name, 'r', encoding='utf-8') as f:
        data = f.readlines()
    cnt = 0
    for line in data:
        key = line[:-4]
        if key in jp_chs:
            if line[0] == '「':
                data[cnt] = '「' + jp_chs[key][1:-1] + '」'+line[-4:]
            else:
                data[cnt] = jp_chs[key] + line[-4:]
        else:
            cnt += 1
            failed_text.append(line)

    with open('output/'+file_name, 'w', encoding='utf-8') as f:
        for line in data:
            f.write(line)
